In [71]:
import pandas as pd
import numpy as np
import os
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Model

In [30]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [31]:
rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.figo_num.value_counts()).reset_index()
        .rename(columns={"index": "label", "figo_num": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="weighted")

mc_baseline = opt.mc_classification_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="weighted"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.3665158371040725
Monte Carlo baseline: 0.36771084757470685


In [125]:
model = GaussianNB()
# model = MultinomialNB()

In [126]:
scoring_default = "f1_weighted"

cv_res = cross_val_score(model, x_df, y_df, cv=KFold(n_splits=5), n_jobs=-1, scoring=scoring_default)
print(cv_res)
print(cv_res.mean())

[0.33606208 0.34521324 0.38135076 0.35977421 0.41618788]
0.3677176332075712
